<a href="https://colab.research.google.com/github/vagiztech/InnovateHealth/blob/main/InnovateHealth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, random_split

In [ ]:
!pip install kaggle
# kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

In [ ]:
# upload kaggle API
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vagizyakupov","key":"6c99cce562128285009f42f9a99465bf"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000
!unzip -q skin-cancer-mnist-ham10000.zip -d data_cancer

100% 5.19G/5.20G [00:44<00:00, 186MB/s]
100% 5.20G/5.20G [00:44<00:00, 125MB/s]


In [ ]:
import os
import shutil

# Пути к файлам и папкам
csv_file = 'data_cancer/HAM10000_metadata.csv'
images_dir = 'data_cancer/HAM10000_images_part_2'

# Чтение CSV-файла
metadata = pd.read_csv(csv_file)

# Перебор строк в DataFrame
for index, row in metadata.iterrows():
    image_name = row['image_id'] + '.jpg'  # предполагаем, что расширение файлов .jpg
    class_name = row['dx']
    source_path = os.path.join(images_dir, image_name)
    target_dir = os.path.join(images_dir, class_name)

    # Проверка наличия файла и перемещение
    if os.path.exists(source_path):
        os.makedirs(target_dir, exist_ok=True)  # Создание папки класса, если не существует
        shutil.move(source_path, os.path.join(target_dir, image_name))


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

full_dataset = ImageFolder(root='/content/data_cancer/HAM10000_images_part_2', transform=transform)
num_classes = len(full_dataset.classes)

train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size
train_dataset, valid_dataset = random_split(full_dataset, [train_size, valid_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [ ]:
def google(): # pretrained=True для tensorflow
    model = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
    model.fc = torch.nn.Linear(1024, num_classes)
    for param in model.parameters():
        param.requires_grad = True
    model.inception3a.requires_grad = False
    model.inception3b.requires_grad = False
    model.inception4a.requires_grad = False
    model.inception4b.requires_grad = False
    model.inception4c.requires_grad = False
    model.inception4d.requires_grad = False
    model.inception4e.requires_grad = False
    return model

In [ ]:
def train(model, optimizer, train_loader, val_loader, epoch=10):
    loss_train, acc_train = [], []
    loss_valid, acc_valid = [], []
    for epoch in tqdm(range(epoch)):
        losses, equals = [], []
        torch.set_grad_enabled(True)

        # Train.
        model.train()
        for i, (image, target) in enumerate(train_loader):
            image = image.to(device)
            target = target.to(device)
            output = model(image)
            loss = criterion(output,target)

            losses.append(loss.item())
            equals.extend(
                [x.item() for x in torch.argmax(output, 1) == target])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_train.append(np.mean(losses))
        acc_train.append(np.mean(equals))
        losses, equals = [], []
        torch.set_grad_enabled(False)

        # Validate.
        model.eval()
        for i , (image, target) in enumerate(valid_loader):
            image = image.to(device)
            target = target.to(device)

            output = model(image)
            loss = criterion(output,target)

            losses.append(loss.item())
            equals.extend(
                [y.item() for y in torch.argmax(output, 1) == target])

        loss_valid.append(np.mean(losses))
        acc_valid.append(np.mean(equals))

    return loss_train, acc_train, loss_valid, acc_valid

In [ ]:
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'

In [ ]:
# ОБУЧЕНИЕ НЕ ТЫКАТЬ
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
model = google()
print('Model: GoogLeNet\n')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model = model.to(device)

loss_train, acc_train, loss_valid, acc_valid = train(
model, optimizer, train_loader, valid_loader, 30)
print('acc_train:', acc_train, '\nacc_valid:', acc_valid)

Model: GoogLeNet



100%|██████████| 30/30 [27:46<00:00, 55.56s/it]

acc_train: [0.6802093718843469, 0.8163010967098704, 0.9132602193419741, 0.9608673978065803, 0.9860418743768694, 0.9920239282153539, 0.9962612163509471, 0.9900299102691924, 0.984297108673978, 0.9937686939182453, 0.9895314057826521, 0.9925224327018943, 0.9927716849451645, 0.9955134596211366, 0.9967597208374875, 0.9962612163509471, 0.9945164506480558, 0.9977567298105683, 0.9967597208374875, 0.9912761714855434, 0.9940179461615155, 0.9957627118644068, 0.9985044865403788, 0.9985044865403788, 0.998753738783649, 0.9912761714855434, 0.9902791625124626, 0.9932701894317049, 0.9967597208374875, 0.9977567298105683] 
acc_valid: [0.7497507477567298, 0.7946161515453639, 0.8344965104685942, 0.8295114656031904, 0.839481555333998, 0.8364905284147557, 0.843469591226321, 0.8354935194416749, 0.8334995014955134, 0.8484546360917248, 0.8354935194416749, 0.8344965104685942, 0.8325024925224327, 0.8454636091724825, 0.8364905284147557, 0.8374875373878365, 0.8444666001994018, 0.839481555333998, 0.8454636091724825, 

In [ ]:
# График после обучения не тыкать
import matplotlib.pyplot as plt

def plot_loss(Loss_train, Loss_val):
    plt.figure(figsize=(12, 5))
    plt.plot(range(len(Loss_train)), Loss_train, color='orange', label='train', linestyle='--')
    plt.plot(range(len(Loss_val)), Loss_val, color='blue', marker='o', label='val')
    plt.legend()
    plt.show()

plot_loss(loss_train, loss_valid)

In [ ]:
# Не тыкать
torch.save(model, '/content/weights2.pt')

In [ ]:
def load_and_preprocess_image(image_path):
    # Загружаем изображение и преобразуем его
    image = Image.open(image_path)
    transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)  # добавляем размерность пакета

    return input_batch

def predict_image_class(image_path, model):
    # Загружаем и предобрабатываем изображение
    input_tensor = load_and_preprocess_image(image_path)

    # Перемещаем тензор на GPU, если доступен
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_tensor = input_tensor.to(device)
    model = model.to(device)

    # Переводим модель в режим оценки (eval)
    model.eval()

    # Проходим изображение через модель
    with torch.no_grad():
        output = model(input_tensor)

    # Получаем предсказание
    _, predicted_class = torch.max(output, 1)

    return predicted_class.item()

In [ ]:
predicted_class_name = full_dataset.classes[:]
print(predicted_class_name)

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [ ]:
weights_path = '/content/weights30.pt'
model = torch.load(weights_path)

In [ ]:
image_path = "/content/data_cancer/HAM10000_images_part_2/nv/ISIC_0029344.jpg"
def predict_class_image(image_path, model=model):
  predicted_class = predict_image_class(image_path, model)
  return predicted_class_name[predicted_class]
print(f"Модель предсказывает класс: {predict_class_image(image_path)}")

Модель предсказывает класс: nv


In [ ]:

def test_model_lul(Patch):
  test_dict = {}
  folder_path = Patch  # Укажите свой путь к папке

  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path):
          predicted_class = predict_image_class(file_path, model)
          pred = predicted_class_name[predicted_class]
          if pred in test_dict.keys():
            test_dict[pred] += 1
          else:
            test_dict[pred] = 0
  return test_dict

In [ ]:
test_model_lul('/content/data_cancer/HAM10000_images_part_2/nv')

{'nv': 3238, 'bkl': 13, 'mel': 16, 'bcc': 2, 'vasc': 0}

In [ ]:
columns = ['pacientID','full name', 'Typeofdisease', 'Cancer']
df_doctor = pd.DataFrame(columns=columns)

# Функция для заполнения следующей строки
def fill_string(values, dataframe):
    new_string = pd.Series(values, index=dataframe.columns)
    dataframe = pd.concat([dataframe, new_string.to_frame().T], ignore_index=True)
    return dataframe

In [ ]:
info_dict = {'akies': ['актинический кератоз/болезнь Боуэна L57.0/D04.9', 'предрак', 'дерматолог'],
             'bcc': ['базалиома D48', 'рак', 'онколог'],
             'bkl': ['доброкачественные поражения, подобные кератозу ', 'ок', 'дома'],
             'df': ['дерматофиброма D23.9', 'ок', 'дерматолог'],
             'mel': ['меланома D48', 'рак', 'онколог'],
             'nv': ['меланоцитарный невус', 'ок', 'дома'],
             'vasc': ['сосудистые поражения', 'ок', 'дома']}

In [ ]:
def pacient(path):
  global df_doctor
  pacientID = input('введите ваш id: ')
  full_name = input('введите ваше ФИО: ')
  disease = predict_class_image(path)
  Typeofdisease = info_dict[disease][0]
  Cancer_num = info_dict[disease][1]
  doctor = info_dict[disease][2]
  if Cancer_num == 'рак':
    Canser = 2
  elif Cancer_num == 'предрак':
    Canser = 1
  else:
    Canser = 0
  df_doctor = fill_string([pacientID, full_name, Typeofdisease, Canser], df_doctor)
  if doctor == 'дерматолог':
    return 'Запишитесь к врачу дерматологу'
  if doctor == 'онколог':
    return 'Запишитесь к врачу онкологу'
  if doctor == 'дома':
    return 'У вас все впорядке !'

In [ ]:
path_pacient = '/content/data_cancer/HAM10000_images_part_2/mel/ISIC_0029913.jpg'
pacient(path_pacient)

введите ваш id: 1234
введите ваше ФИО: Иванов Иван Иванович


'Запишитесь к врачу онкологу'

In [ ]:
print(df_doctor)

  pacientID             full name Typeofdisease Cancer
0      1234  Иванов Иван Иванович  меланома D48      2


In [ ]:
import torch.nn.functional as F
# Функция для классификации изображения

def classify_image(image_path, model):
    input_tensor = load_and_preprocess_image(image_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_tensor = input_tensor.to(device)
    # Передача изображения через модель
    with torch.no_grad():
        output = model(input_tensor)

    # Получение предсказанного класса
    _, predicted_class = torch.max(output, 1)
    probabilities = F.softmax(output, dim=1)

    return predicted_class.item(), probabilities
# Загрузка обученной модели
weights_path = '/content/weights30.pt'
model = torch.load(weights_path)

# Путь к папке с изображениями для классификации
folder_path = '/content/data_cancer/ham10000_images_part_1'

predicted_class_name = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
# Создание списка предсказаний
predictions = []

# Классификация изображений в папке
for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    if os.path.isfile(image_path):
        predicted_class, probabilities = classify_image(image_path, model)
        image_id = filename.split('.')[0]
        predictions.append((image_id, predicted_class_name[predicted_class]))

# Создание DataFrame из списка предсказаний
predictions_df = pd.DataFrame(predictions, columns=['image_id', 'predict'])


In [ ]:
metadata = pd.read_csv(csv_file)

In [ ]:
# Объединение с исходным DataFrame 'metadata' по столбцу 'image_id'
metadata = metadata.merge(predictions_df, on='image_id', how='left')
# Очистка от NaN
metadata_cleaned = metadata.dropna(subset=['dx', 'predict'])

# Добавление столбца 'is_True'
metadata_cleaned['is_True'] = (metadata_cleaned['dx'] == metadata_cleaned['predict']).astype(int)

# Создание нового DataFrame 'result'
result = metadata_cleaned[['dx', 'predict', 'is_True']].copy()


<ipython-input-29-0467e8446385>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_cleaned['is_True'] = (metadata_cleaned['dx'] == metadata_cleaned['predict']).astype(int)


In [ ]:
statistics = result['is_True'].value_counts()
total_rows = len(result)
is_True_result = (statistics.iloc[1]/(statistics.iloc[0]+statistics.iloc[1])) * 100
print(f'Модель на не зависимых данных cделала  {(100 - round(is_True_result,2))} % верных прогнозов.')

Модель на не зависимых данных cделала  82.62 % верных прогнозов.


In [ ]:
!python --version

Python 3.10.12
